# Warp Algorithm for Regression

This extension implements a version of the Warp algoirthm that detects regressions and adapts to them.

Here, we have used the error_between_line_regression and error_within_line_regression functions in the correction.py file to test the algorithm against both within-line-regression and between-line-regression.

In [ ]:
# open the generated synthetic data file and read it as a python dictionary called data

import json

file = open('synthetic_dataset_with_time.txt', 'r')
text = file.read()
data = json.loads(text)

In [ ]:
import emip_toolkit as EMTK

# find areas of interest (AOIs) once again
image_path = "./"
image = "synthetic_text.png"
aoi = EMTK.find_aoi(image, image_path, level="sub-line")


# add tokens to AOIs 
file_path = "./"
aois_with_tokens = EMTK.add_tokens_to_AOIs(file_path, image.split(".")[0]+".txt", aoi)
aois_with_tokens.head()


import correction

# find the y coordinate of each line in the text
line_ys = correction.find_lines_Y(aoi)

In [ ]:
import numpy as np
import drift_algorithms as algo
from tqdm import tqdm

In [ ]:
# draw original correct fixations

correction.draw_fixation('synthetic_text.png', data['robot1'].copy())

In [ ]:
# generate within-line regression over the correct trial
image_path = "./"
image = "synthetic_text.png"

aoi_words = EMTK.find_aoi(image, image_path, level="sub-line")
aoi_lines = EMTK.find_aoi(image, image_path, level="line")

# Creating a sample lsit to store an array of an array of words where each index represents a line
words_sorted_by_line = []

for i in range(len(aoi_lines)):
    words_sorted_by_line.append([])

# Looping through the words and appending them to the correct line
for i in range(len(aoi_words)):
    word_line_number = int(aoi_words.at[i, 'name'].strip().split(' ')[1])
    # Appending the x and y coordinates of the word to the correct line
    words_sorted_by_line[word_line_number-1].append([aoi_words.at[i, 'x'], aoi_words.at[i, 'y']])

error_test, data['robot1'] = correction.error_within_line_regression(1, data['robot1'], words_sorted_by_line, aoi_lines)

# Printing the length of robot1 and error_test to check if the length of the two lists are the same
print(len(data['robot1']))
print(len(error_test))

# draw the trial with added error
correction.draw_fixation('synthetic_text.png', error_test)